In [1]:
# temp_rf.py
# initial author: Tim Currier; last updated: 2022-05-25
# saves numpy arrays and 2-color mp4 movies of the STRFs for all ROIs in a set
# array filter is 4 sec; movie of the first 2 sec at 1/2 real-time speed

# argumenmts: [1] date (yyyy-mm-dd); [2] series_number; [3] roi_set_name
# implementation: save_strfs.py 2022-03-17 1 roi_set_postfrom visanalysis.analysis import imaging_data, shared_analysis
from visanalysis.util import plot_tools
import matplotlib.pyplot as plt
import numpy as np
import os
from two_photon_analysis import medulla_analysis as ma
import sys
import cv2
import warnings
from tifffile import imsave
from pathlib import Path 
from scipy.ndimage import gaussian_filter

In [26]:
# define recording series to analyze
experiment_file_directory = '/Volumes/ROG2TBAK/data/bruker/20221025'
experiment_file_name = '2022-10-25' #sys.argv[1]
series_number = '1' #int(sys.argv[2])
roi_set_name = 'proximal_z2' #sys.argv[3]

# join path to proper format for ImagingDataObject()
file_path = os.path.join(experiment_file_directory, experiment_file_name + '.hdf5')
print(file_path)

# create save directory
save_directory = '/Volumes/ROG2TBAK/data/bruker/trfs' + experiment_file_name + '/'
Path(save_directory).mkdir(exist_ok=True)

# create ImagingDataObject (wants a path to an hdf5 file and a series number from that file)
ID = imaging_data.ImagingDataObject(file_path,
                                    series_number,
                                    quiet=True)

# get ROI timecourses and stimulus parameters
roi_data = ID.getRoiResponses(roi_set_name);
epoch_parameters = ID.getEpochParameters();
run_parameters = ID.getRunParameters();

/Volumes/ROG2TBAK/data/bruker/20221025/2022-10-25.hdf5


In [38]:
# Stimulus recreation testing

# from flystim.distribution.py:
class Ternary:
    def __init__(self, rand_min, rand_max):
        self.rand_min = rand_min
        self.rand_max = rand_max

    def get_random_values(self, output_shape):
        rand_values = np.random.choice([self.rand_min, (self.rand_min + self.rand_max)/2, self.rand_max], size=output_shape)
        return rand_values
    
# from flystim.stimuli.py:
noise_distribution = getattr(distribution, distribution_data['name'])(*distribution_data.get('args', []), **distribution_data.get('kwargs', {}))

distribution_data = {'name': 'Ternary',
                     'args': [],
                     'kwargs': {'rand_min': 0,
                                'rand_max': 1}}

color = noise_distribution.get_random_values(1)

NameError: name 'distribution' is not defined

In [43]:
# This is just a debugging cell; these values are normally defined below:
update_rate = 4
rand_min = 0
rand_max = 1
stim_time = 3
n_frames = update_rate*stim_time
num_epochs = 5
seed = 1
np.random.seed(1)
idle_color = .1111


# initialize array that will contain stimuli for all trials
all_stims = np.zeros((int(n_frames),int(num_epochs)))
print(all_stims)
# populate all-trial stimulus array
for trial_num in range(1, int(num_epochs+1)):
    # pull start_seed for trial
    # initialize stimulus frames variable with full idle color
    stim = np.full(int(n_frames),idle_color)
    print(stim)
    # populate stim array (T) specifically during "stim time"
    for stim_ind in range(0, stim.shape[0]):
        # find time in sec at stim_ind
        #t = stim_ind/update_rate;
        # define seed at each timepoint
        #seed = int(round(start_seed + t*update_rate))
        np.random.seed(seed)
        # find random values for the current seed and write to pre-initialized stim array
        rand_values = np.random.choice([rand_min, (rand_min + rand_max)/2, rand_max], 1);
        stim[stim_ind]= rand_values
        seed = seed+1
            
    # save trial stimulus to all_stims(Height, Width, Time, Trial)
    all_stims[:,(trial_num-1)] = stim;


[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111
 0.1111 0.1111]
[0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111
 0.1111 0.1111]
[0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111
 0.1111 0.1111]
[0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111
 0.1111 0.1111]
[0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111 0.1111
 0.1111 0.1111]


In [44]:
print(all_stims.shape)
all_stims

(12, 5)


array([[0.5, 1. , 0. , 0. , 1. ],
       [0. , 0. , 0.5, 0.5, 0. ],
       [1. , 0. , 0. , 0.5, 1. ],
       [1. , 0.5, 0.5, 1. , 0.5],
       [1. , 0.5, 0.5, 0. , 0.5],
       [1. , 1. , 0.5, 1. , 0.5],
       [0. , 0.5, 1. , 0. , 0.5],
       [0. , 1. , 0.5, 0. , 0.5],
       [1. , 0.5, 0. , 1. , 1. ],
       [0.5, 0.5, 0.5, 0.5, 0. ],
       [0.5, 1. , 0.5, 1. , 0.5],
       [1. , 1. , 0.5, 0. , 0.5]])

In [ ]:
# pull run parameters (same across all trials)
update_rate = run_parameters['update_rate']
rand_min = run_parameters['rand_min']
rand_max = run_parameters['rand_max']

# calculate size of noise grid, in units of patches (H,W)
#output_shape = (int(np.floor(run_parameters['grid_height']/run_parameters['patch_size'])), int(np.floor(run_parameters['grid_width']/run_parameters['patch_size'])));

# calculate number of time points needed
n_frames = update_rate*(run_parameters['stim_time']);

# initialize array that will contain stimuli for all trials
all_stims = np.zeros(int(n_frames) ,int(run_parameters['num_epochs']))
# populate all-trial stimulus array
for trial_num in range(1, int(run_parameters['num_epochs']+1)):
    # pull start_seed for trial
    start_seed = epoch_parameters[(trial_num-1)]['start_seed']
    # initialize stimulus frames variable with full idle color
    stim = np.full(int(n_frames),run_parameters['idle_color'])
    # populate stim array (T) specifically during "stim time"
    for stim_ind in range(0, stim.shape[0]):
        # find time in sec at stim_ind
        t = stim_ind/update_rate;
        # define seed at each timepoint
        seed = int(round(start_seed + t*update_rate))
        np.random.seed(seed)
        # find random values for the current seed and write to pre-initialized stim array
        rand_values = np.random.choice([rand_min, (rand_min + rand_max)/2, rand_max], 1);
        stim[stim_ind]= rand_values
            
    # save trial stimulus to all_stims(Height, Width, Time, Trial)
    all_stims[:,(trial_num-1)] = stim;

In [ ]:
# define filter length in seconds, convert to samples
filter_length = 8;
filter_len = filter_length*run_parameters['update_rate'];

# iterate over ROIs to save STRFs and movies
print('Calculating STRFs...')
for roi_id in range(0, roi_data['epoch_response'].shape[0]):
    # initialize strf by trial array (H,W,T,Tr)
    roi_strf = np.zeros(output_shape+(int(filter_len),int(run_parameters['num_epochs'])));
    for trial_num in range(0, int(run_parameters['num_epochs'])):
        current_resp = roi_data['epoch_response'][roi_id,trial_num]
        # initialize strf and full time series for update rate of stimulus
        full_t = np.arange(run_parameters['pre_time'],run_parameters['stim_time'] + run_parameters['pre_time'],1 / run_parameters['update_rate'])
        strf = np.zeros(output_shape+(int(filter_len),))
        # linearly interpolate response to match stimulus timing
        full_resp = np.interp(full_t,roi_data['time_vector'],current_resp)
        resp_mean = np.mean(full_resp)
        resp_var = np.var(full_resp)
        # compute TRF for mean-subtracted stimulus and response; then compile STRF across patches
        n=all_stims.shape[2];
        ext_size=2*n-1
        fsize=2**np.ceil(np.log2(ext_size)).astype('int')
        for phi in range(0,output_shape[0]):
            for theta in range(0,output_shape[1]):
                patch_stim = all_stims[phi,theta,:,trial_num];
                patch_mean = np.mean(patch_stim)
                filter_fft = np.fft.fft(full_resp-resp_mean,fsize) * np.conj(np.fft.fft(patch_stim-patch_mean,fsize));
                filt = np.real(np.fft.ifft(filter_fft))[0:int(filter_len)];
                trf = np.flip(filt);
                strf[phi,theta,:] = trf;
        # add trial strf to roi_strf array
        roi_strf[:,:,:,trial_num] = strf;
    # compute mean STRF
    roi_mean_strf = np.mean(roi_strf,3);
    # flip again to have t=0 at beginning
    roi_mean_strf = np.flip(roi_mean_strf,2)
    # calculate std for each patch
    STRF_std = np.std(roi_strf,(2,3))
    # divide by std to generate z-scored STRF
    roi_mean_strf_z = np.zeros(roi_mean_strf.shape)
    for frame in range(0,roi_mean_strf.shape[2]):
        roi_mean_strf_z[:,:,frame] = roi_mean_strf[:,:,frame]/STRF_std
    # print(roi_mean_strf_z[:,:,1])
    # save mean, dc-subtracted (AND a z-scored) STRF
    np.save('/Volumes/TimBigData/Bruker/STRFs/' + experiment_file_name + '/' + experiment_file_name + '-' + roi_set_name + '_' + str(roi_id) + '_STRF.npy', roi_mean_strf)
    np.save('/Volumes/TimBigData/Bruker/STRFs/' + experiment_file_name + '/' + experiment_file_name + '-' + roi_set_name + '_' + str(roi_id) + '_STRFz.npy', roi_mean_strf_z)
    # oversample z-scored STRF in x and y so video looks better
    big_strf=roi_mean_strf_z.repeat(20,axis=0)
    bigger_strf=big_strf.repeat(20,axis=1)
    # oversample z-scored STRF in t so framerate can be reduced
    biggest_strf=bigger_strf.repeat(2,axis=2)
    # convert z-scored STRF to -1 to 1 scale (units are standard deviations)
    low_lim = -3
    high_lim = 3
    new_strf = ((biggest_strf - low_lim) * (2/(high_lim - low_lim))) - 1
    new_strf=np.where(new_strf>1,1,new_strf)
    new_strf=np.where(new_strf<-1,-1,new_strf)
    # make empty rgb array and populate with positive or negative values
    rgb_strf=np.zeros((new_strf.shape+(3,)))
    pos_strf=np.where(new_strf>0,new_strf,0)
    neg_strf=np.where(new_strf<0,new_strf*-1,0)
    rgb_strf[:,:,:,0]=1-(pos_strf*1)-(neg_strf*.3)
    rgb_strf[:,:,:,2]=1-(pos_strf*1)-(neg_strf*0)
    rgb_strf[:,:,:,1]=1-(pos_strf*.3)-(neg_strf*1)
    rgb_strf=np.where(rgb_strf>1,1,rgb_strf)
    # scale rgb_strf to 0-255
    rgb_strf = (rgb_strf*255).astype('uint8')
    # save multicolor video
    fps = 10
    video = cv2.VideoWriter('/Volumes/TimBigData/Bruker/STRFs/' + experiment_file_name + '/' + experiment_file_name + '-' + roi_set_name + '_' + str(roi_id) + '_movie.mp4', cv2.VideoWriter_fourcc(*'mp4v'), float(fps), (new_strf.shape[1],new_strf.shape[0]))
    for frame_count in range(int(new_strf.shape[2]/2)):
        img = rgb_strf[:,:,frame_count,:]
        video.write(img)
    video.release()
print('Done.')